In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
fileList = ["12-30-2022sequences.txt", "12-30-salk.txt", "12-30not_clones.txt", "lotsOfSequences.txt"]

rightList = []
leftList = []
for thisFile in fileList:

    #opens the file and turns it into a list of strings where
    #each item is a line in the file
    fhand = open(thisFile)
    flist = fhand.readlines()
    flist = [line.strip() for line in flist]

    # makes list of positions where there is a "COMMENT" section in the file.
    # this is important becuase comment sections contain locus and position of sequence
    # relative to insertion (right or left border)
    linecount = 0
    commentLines = []
    for line in flist:
        if line[0:7] == "COMMENT":
            commentLines.append(linecount)
        linecount += 1

    # goes to each comment section and extracts locus, right or left border position,
    # and the whole sequence itself
    seqList = []
    lcount = 0
    for com in commentLines:
        border = ''
        gene = ''
        commentrl = flist[com]
        commentgene = flist[com+1].split()
        gene = commentgene[-1]
        if "right" in commentrl:
            border = "R"
        if "left" in commentrl:
            border = "L"
            lcount += 1
        seqString = ''
        n = com
        for liney in flist[com:com+75]:
            n += 1
            if liney == "ORIGIN":
                while flist[n] != "//":
                    n += 1
                    seqString = seqString + flist[n]
        seqStringList = [i for i in seqString]
        newSeqList = []
        addSeq = True
        #ensures that sequences only contain a, g, c, t
        for char in seqStringList:
            if char == 'a' or char == 'c' or char == 'g' or char == 't':
                newSeqList.append(char)
            else:
              if char.isalpha():
                addSeq = False
                break
        if len(newSeqList) < 80:
          addSeq = False
        if addSeq:
            seqList.append([gene, border, newSeqList])

    # reformats each sequence so that it includes the 80 closest bases to the insertion
    count = 0
    for i in seqList:
        if i[1] == 'L':
            i[2].append("1")
            leftList.append(i[2][-81:])
        if i[1] == 'R':
            i[2][80] = 1
            rightList.append(i[2][:81])



print("R:", len(rightList))
print("L:", len(leftList))

#ensures that there are no duplicates in the final list
count = 0
for i in rightList:
    for n in rightList:
        if i == n:
            rightList.remove(i)
            count += 1

count = 0
for i in leftList:
    for n in leftList:
        if i == n:
            leftList.remove(i)
            count += 1


print("R:", len(rightList))
print("L:", len(leftList))

R: 467
L: 726
R: 157
L: 244


In [ ]:
dfR = pd.DataFrame(rightList)
dfR.head()



,0,1,2,3,4,5,6,7,8,9,...,71,72,73,74,75,76,77,78,79,80
0,g,g,g,g,a,t,c,a,g,t,...,g,a,a,t,c,t,a,t,c,1
1,g,g,g,t,a,c,c,c,g,g,...,t,g,a,t,c,a,t,a,t,1
2,a,t,c,t,g,g,a,a,a,t,...,t,t,t,a,g,c,a,a,a,1
3,c,a,g,a,t,t,g,t,c,g,...,a,a,g,g,t,t,t,t,c,1
4,a,g,g,g,t,a,c,c,c,g,...,t,t,c,g,a,a,a,a,g,1


In [ ]:
#reads file containing whole Arabidopsis genome and converts it into a list of lists where
#each line in the file is an item in the fgenome list
fhandle = open('AT_genome_SEQUENCE.txt')
fgenome = fhandle.readlines()
fgenome = [line.strip() for line in fgenome]
fgenome = [[char.lower() for char in line] for line in fgenome]

#ensures that empty lines in the file are not included in the fgenome list
emptyList = ['']
fgenome = [i for i in fgenome if i != emptyList]

#adds "0" to the end of each item in fgenome
#serves as label for binary classification
for i in range(len(fgenome)):
    fgenome[i].append("0")

print(len(fgenome))

FileNotFoundError: ignored

In [ ]:
#import scipy.stats as stats


def GCcont(seq):
    ''' finds the percent GC content of a sequence as a decimal

        seq: list of bases in sequence, in lowercase. assumes there is an extra
        character at the end denoting the label (in this case, 0 or 1)
    '''
    count = 0
    for base in seq:
        if base == 'g' or base == 'c':
            count += 1
    return (count/(len(seq)-1))

def genNegList(borderList, fgenome):
  ''' generates a list of 'no insertion' sequences, AKA negative cases, such that
      the GC content distribution of the generated list is not statistically different
      from that of a specified list (borderList). Distributions are compared using
      a Mann-Whitney U test.

      borderList: list of positive cases, generated list will have GC content distribution
                  similar to that of borderList. Additionally, generated list will be of
                  same length as borderList

      fgenome: list of sequences that will be randomly sampled to generate list
               of negative cases

  '''

  GClist1 = []
  GClist0 = []

  #generates list of GC contents for borderList to be compared to NegList later
  for item in borderList:
    GClist1.append(GCcont(item))


  pvalue = 0
  while pvalue < 0.4:
      pvalue = 0
      NegList = []
      GClist0 = []
      while len(NegList) < len(borderList):
          #randomly selects an index from fgenome
          x = random.randint(0, len(fgenome)-1)
          #ensures that the item of selected index from fgenome is a sequence
          if (fgenome[x][0] == 'g' or fgenome[x][0] == 'c' or fgenome[x][0] == 't' or fgenome[x][0] == 'a') and (len(fgenome[x]) == 81):
              #adds the GC content of selected item from fgenome to a copy of GClist0
              #and tests to ensure that distributions are not made statistically different
              #by adding this value
              GClist0copy = GClist0.copy()
              GClist0copy.append(GCcont(fgenome[x][0:79]))
              ustat, pvalue = stats.mannwhitneyu(GClist0copy, GClist1)
              #if added value does not make distributions significantly different,
              #it is added to NegList
              if pvalue > 0.4:
                  NegList.append(fgenome[x])
                  GClist0.append(GCcont(fgenome[x][0:79]))
      #p-value of final generated NegList is tested to ensure it is above 0.4
      ustat, pvalue = stats.mannwhitneyu(GClist0, GClist1)
      #prints p-value of final generated list
      print(pvalue)

  #returns generated list of negative cases
  return NegList


#generates list of negative cases for right and left lists
rightNegList = genNegList(rightList, fgenome)
leftNegList = genNegList(leftList, fgenome)

0.40880884683881025
0.40685765417495623


In [ ]:
#combines positive and negative cases (insertion and no insertion) into one final list
CombinedRightList = rightList + rightNegList
CombinedLeftList = leftList + leftNegList


#ensures all items in left list are of appropriate length
line = 0
for i in CombinedLeftList:
    if len(i) != 81:
        CombinedLeftList.remove(i)
        print("(left list) wrong length on item", line)
        line+=1

#ensures that all items in right list are of appropriate length
line = 0
for i in CombinedRightList:
    if len(i) != 81:
        CombinedRightList.remove(i)
        print("(right list) wrong length on item", line)
        line+=1

#prints the length of each final list
print("R:", len(CombinedRightList))
print("L:", len(CombinedLeftList))

#writes right list to .csv file
dfR = pd.DataFrame(CombinedRightList)
dfR.to_csv(input("file name: "))

#writes left list to .csv file
dfL = pd.DataFrame(CombinedLeftList)
dfL.to_csv(input("file name: "))

R: 314
L: 488
file name: agroRight.csv
file name: agroLeft.csv
